# Fetch data from wekeo
---

First we need to complete the following request too get the authentication token. 

__$ curl --request GET --header 'authorization: Basic <credentials>' https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/gettoken__


Next we have to accept the terms and conditions using the authentication token.

__$ curl --request PUT --header 'accept: application/json' --header 'authorization: <access_token>' --data 'accepted=true' https://'wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/termsaccepted/Copernicus_General_License__

Ultimately we can get the subsetted dataset via the following command.

__$ curl --request GET --header 'authorization: <access_token>' https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/querymetadata/<datasetId>__

In [59]:
import requests
from config import credentials

In [60]:
url_token = "https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/gettoken"
res_token = requests.get(url_token, auth=credentials)
print("The request token is: ", res_token.json()["access_token"])

The request token is:  19e23327-7b1d-3b3d-a7a9-0a7928012ac1


In [34]:
url_tc = "https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/termsaccepted/Copernicus_General_License"
res_tc = requests.get(url_tc, headers={"accept":"application/json", "authorization":res_token.json()["access_token"]})

In this example we created an API call at wekeo.eu. We selected the area around Reading. 

The API request looks like this:

{
  "datasetId": "EO:ESA:DAT:SENTINEL-2:MSI",
  "boundingBoxValues": [
    {
      "name": "bbox",
      "bbox": [
        -1.0049839613742617,
        51.41492053337421,
        -0.9518149873018946,
        51.43582605943038
      ]
    }
  ],
  "dateRangeSelectValues": [
    {
      "name": "position",
      "start": "2015-10-01T00:00:00.000Z",
      "end": "2021-11-10T00:00:00.000Z"
    }
  ],
  "stringChoiceValues": [
    {
      "name": "processingLevel",
      "value": "LEVEL1C"
    }
  ]
}

In [35]:
dataset_id = "EO:ESA:DAT:SENTINEL-2:MSI"
url_dataquery = "https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/querymetadata/"
res_dataquery = requests.get(url_dataquery+dataset_id, headers={"authorization":res_token.json()["access_token"]})
dataquery = res_tc.json()

In [55]:
data = {
  "datasetId": "EO:ESA:DAT:SENTINEL-2:MSI",
  "boundingBoxValues": [
    {
      "name": "bbox",
      "bbox": [
        -1.0049839613742617,
        51.41492053337421,
        -0.9518149873018946,
        51.43582605943038
      ]
    }
  ],
  "dateRangeSelectValues": [
    {
      "name": "position",
      "start": "2015-10-01T00:00:00.000Z",
      "end": "2021-11-10T00:00:00.000Z"
    }
  ],
  "stringChoiceValues": [
    {
      "name": "processingLevel",
      "value": "LEVEL1C"
    }
  ]
}

In [56]:
res_post = requests.post(url_dataquery, headers={"authorization":res_token.json()["access_token"], "content-type":"application/json"}, data=data)

In [57]:
res_post.json()

{'detail': 'The requested URL was not found on the server.  If you entered the URL manually please check your spelling and try again.',
 'status': 404,
 'title': 'Not Found',
 'type': 'about:blank'}

In [53]:
type(data)

dict